In [1]:
#generating dataset Haarcascade
import cv2

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # scaling factor = 1.3
        # minimum neighbor = 5
        
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)  # Change to 0 if you're using the default webcam
    id = 1
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        cropped_face = face_cropped(frame)
        if cropped_face is not None:
            img_id += 1
            face = cv2.resize(cropped_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow("Cropped face", face)
        
        if cv2.waitKey(1) == 13 or int(img_id) == 200:  # 13 is the ASCII character of Enter
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

generate_dataset()

In [2]:
#training the data using LBPH
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy
 
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
     
    faces = []
    ids = []
     
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
         
        faces.append(imageNp)
        ids.append(id)
         
    ids = np.array(ids)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("data")

In [3]:
#training the data using knn
import os
import face_recognition
import numpy as np

def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []
    
    for image in path:
        img = face_recognition.load_image_file(image)
        face_encodings = face_recognition.face_encodings(img)
        if len(face_encodings) > 0:
            face_encoding = face_encodings[0]
            id = int(os.path.split(image)[1].split(".")[1])
            faces.append(face_encoding)
            ids.append(id)
    
    ids = np.array(ids)
    
    # Here, you would train your classifier using the face encodings.
    # face_recognition library doesn't directly provide a method to save a classifier.
    # You might need to use another machine learning library to train a classifier on these encodings.
    # Example using K-Nearest Neighbors (KNN)
    
    from sklearn import neighbors
    n_neighbors = int(round(np.sqrt(len(faces))))
    
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm='ball_tree', weights='distance')
    knn_clf.fit(faces, ids)
    
    # Save the trained KNN classifier
    import pickle
    with open("knn_classifier.clf", 'wb') as f:
        pickle.dump(knn_clf, f)

train_classifier("data")

In [3]:
#recogning the face using LPBH
import cv2
import numpy as np
from PIL import Image
import os

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        
        id, pred = clf.predict(gray_img[y:y+h, x:x+w])
        confidence = int(100 * (1 - pred / 300))
        
        if confidence > 75:
            if id == 1:
                cv2.putText(img, "Milan", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
    
    return img

# Load classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    print("Error: Could not open video capture device.")
    exit()

while True:
    ret, img = video_capture.read()
    if not ret:
        print("Error: Failed to grab frame.")
        break
    
    img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
    cv2.imshow("Face Detection", img)
    
    # Check if the 'Esc' key (key code 27) is pressed
    if cv2.waitKey(1) == 27:  # 27 is the ASCII code for the 'Esc' key
        print("Esc key pressed. Exiting...")
        break

    # Check if the window's close (X) button is clicked
    if cv2.getWindowProperty("Face Detection", cv2.WND_PROP_VISIBLE) < 1:
        print("Window closed. Exiting...")
        break

video_capture.release()
cv2.destroyAllWindows()

Window closed. Exiting...
